In [1]:
# 定数・変数の定義
from enum import Enum

# 状態のEnumの定義
class Status(Enum):
    WAIT = 1  # SYNコードの受付状態
    READY = 2 # 入力受付状態
    
# 初期状態はwait 
status = Status.WAIT

# SYNコードの定義
SYN_CODE = np.array([0,0,0,1,0,1,1,0])

# 入力データの準備 (SYN + a + SYN)
a_code = np.array([0,1,1,0,0,0,0,1])
b_code = np.array([0,1,1,0,0,0,1,0])
c_code = np.array([0,1,1,0,0,0,1,1])
input_data = np.r_[a_code, SYN_CODE, b_code, SYN_CODE, c_code]

# 入力データを一時的に保持するためのバッファ
bin_data_buffer = np.zeros(8, np.int8)

# メッセージを受信するためのバッファ
input_bin_data = np.empty(0, np.int8)

In [2]:
# キャラクター同期のメッセージ取得手順
# 入力データを1ビットずつ処理する
for signal in input_data:
    
    # 要素を1つずらす [1, 2, 3, 4] → [2, 3, 4, 1]
    bin_data_buffer = np.roll(bin_data_buffer, -1)
    
    # 最新のデータを末尾に代入し、バッファの更新
    bin_data_buffer[-1] = signal
    
    # 待ち状態の場合
    if status == Status.WAIT:
        
        # 直近8ビットのデータがSYNコードと一致するか判定
        if np.array_equal(bin_data_buffer, SYN_CODE):
            
            # SYNコードの場合入力受付状態に変更
            status = Status.READY
        
    # 入力受付状態の場合
    elif status == Status.READY:
        
        # メッセージ内容を保持
        input_bin_data = np.append(input_bin_data, signal)
        
        # SYNコードか判定
        if  np.array_equal(bin_data_buffer, SYN_CODE):
            
            # 直近8ビットがsynコードなので入力データから除外
            input_bin_data = input_bin_data[:-8]
            
            status = Status.WAIT
        
# bin_data_buffer
print("受信データ: {}".format(input_bin_data)) 

受信データ: [0 1 1 0 0 0 1 0]
